# Question

Plotting the mulitpose results shows that as we increase the number of poses included for the randomly split, sorted by RMSD results, they get worse. this shouldn't be possible so i think something must be wrong

# Imports

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from pydantic import BaseModel, Field
import abc
from tqdm import tqdm

## Paths

In [ ]:
data_path = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/20240424_multi_pose_docking_cross_docking/")
csvs_path = data_path / ("results_csvs")
result_csv = csvs_path / "20240503_combined_results_with_data.csv"

In [ ]:
output_data_path = Path("analyzed_data")

In [ ]:
result_csv.exists()

## Load Data

In [ ]:
df = pd.read_csv(result_csv, index_col=0)
df["Complex_ID"] = df["Query_Ligand"] + "_" + df["Reference_Structure"]

# only keep first dataset split

In [ ]:
dates = df.Reference_Structure_Date.unique()
dates.sort()
earliest = dates[:45]

In [ ]:
df_split = df[df.Reference_Structure_Date.isin(earliest)]

In [ ]:
df_split.Reference_Structure_Date.nunique()

In [ ]:
poses1 = df_split.sort_values("Pose_ID", ascending=True).groupby(["Query_Ligand", "Reference_Ligand"]).head(1)

In [ ]:
poses20 = df_split.sort_values("Pose_ID", ascending=True).groupby(["Query_Ligand", "Reference_Ligand"]).head(20)

In [ ]:
poses50 = df_split.sort_values("Pose_ID", ascending=True).groupby(["Query_Ligand", "Reference_Ligand"]).head(50)

In [ ]:
poses1_scored = poses1.sort_values("RMSD", ascending=True).groupby(["Query_Ligand"]).head(1)

In [ ]:
poses20_scored = poses20.sort_values("RMSD", ascending=True).groupby(["Query_Ligand"]).head(1)

In [ ]:
poses50_scored = poses50.sort_values("RMSD", ascending=True).groupby(["Query_Ligand"]).head(1)

In [ ]:
poses1_scored["RMSD"].apply(lambda x: x <= 2.0).sum() / 205

In [ ]:
poses20_scored["RMSD"].apply(lambda x: x <= 2.0).sum() / 205

In [ ]:
poses50_scored["RMSD"].apply(lambda x: x <= 2.0).sum() / 205

# what about if we use the posit score?

In [ ]:
poses1_scored_posit = poses1.sort_values("docking-confidence-POSIT", ascending=False).groupby(["Query_Ligand"]).head(1)

In [ ]:
poses20_scored = poses20.sort_values("docking-confidence-POSIT", ascending=False).groupby(["Query_Ligand"]).head(1)

In [ ]:
poses50_scored = poses50.sort_values("docking-confidence-POSIT", ascending=False).groupby(["Query_Ligand"]).head(1)

In [ ]:
poses1_scored_posit["RMSD"].apply(lambda x: x <= 2.0).sum() / 205

In [ ]:
poses20_scored["RMSD"].apply(lambda x: x <= 2.0).sum() / 205

In [ ]:
poses50_scored["RMSD"].apply(lambda x: x <= 2.0).sum() / 205

# Why are these so different?

In [ ]:
import plotly.express as px

In [ ]:
px.scatter(poses1_scored, x="RMSD", y="docking-confidence-POSIT")

In [ ]:
px.scatter(poses1_scored_posit, x="RMSD", y="docking-confidence-POSIT")